In [1]:
# 라이브러리 임포트
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import json
import os
from datetime import datetime
import numpy as np

print("="*80)
print("HTP 심리검사 해석 모델 테스트 시작")
print("="*80)

c:\Users\helen\anaconda3\envs\doclayout_yolo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HTP 심리검사 해석 모델 테스트 시작


In [2]:
# 설정
MODEL_PATH = "./qwen2.5-htp-layer-freeze-final"  # 학습된 모델 경로
TEST_DATA_PATH = "HTP_data.jsonl"  # 테스트 데이터 경로
OUTPUT_DIR = "./test_results"  # 결과 저장 경로

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"모델 경로: {MODEL_PATH}")
print(f"데이터 경로: {TEST_DATA_PATH}")
print(f"결과 저장: {OUTPUT_DIR}")

모델 경로: ./qwen2.5-htp-layer-freeze-final
데이터 경로: HTP_data.jsonl
결과 저장: ./test_results


## 1. 모델 및 토크나이저 로드

In [3]:
print("\n[1/5] 모델 로드 중...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_PATH,
        trust_remote_code=True,
        padding_side="right",
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"✅ 모델 로드 완료: {MODEL_PATH}")
    print(f"   Device: {model.device}")
    print(f"   VRAM: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("   모델 경로를 확인하세요.")
    raise

`torch_dtype` is deprecated! Use `dtype` instead!



[1/5] 모델 로드 중...
✅ 모델 로드 완료: ./qwen2.5-htp-layer-freeze-final
   Device: cuda:0
   VRAM: 3.09 GB
✅ 모델 로드 완료: ./qwen2.5-htp-layer-freeze-final
   Device: cuda:0
   VRAM: 3.09 GB


## 2. 추론 함수 정의

In [4]:
def generate_htp_interpretation(instruction, image_caption, max_tokens=256, verbose=False):
    """
    HTP 해석 생성 함수
    
    Args:
        instruction: 해석 요청 지시문
        image_caption: 이미지 캡션 (HTP 그림 설명)
        max_tokens: 최대 생성 토큰 수
        verbose: 상세 출력 여부
    
    Returns:
        생성된 심리학적 해석 텍스트
    """
    messages = [
        {
            "role": "system",
            "content": "You are an expert psychologist specialized in HTP (House-Tree-Person) test interpretation. "
                      "Provide detailed and insightful psychological analysis based on the given image caption."
        },
        {
            "role": "user",
            "content": f"{instruction}\n\nImage caption: {image_caption}"
        }
    ]
    
    # 프롬프트 생성
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    if verbose:
        print(f"\n[Prompt (처음 200자)]:\n{prompt[:200]}...\n")
    
    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.8,
            top_p=0.95,
            top_k=40,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 디코딩 (새로 생성된 부분만)
    input_length = inputs['input_ids'].shape[1]
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()

print("✅ 추론 함수 정의 완료")

✅ 추론 함수 정의 완료


## 3. 고정 테스트 케이스 평가

In [5]:
print("\n[2/5] 고정 테스트 케이스 평가 중...")

test_cases = [
    {
        "instruction": "Please provide a psychological interpretation of the following HTP test image caption",
        "input": "The tree is dominant and tall with many branches.",
        "expected_theme": "자신감, 성장 의지, 리더십",
        "category": "tree"
    },
    {
        "instruction": "Please provide a psychological interpretation of the following HTP test image caption",
        "input": "A small house with no windows, located far from the center.",
        "expected_theme": "내향성, 소통 회피, 고립감",
        "category": "house"
    },
    {
        "instruction": "Please provide a psychological interpretation of the following HTP test image caption",
        "input": "The person is drawn very small and placed at the bottom corner.",
        "expected_theme": "낮은 자존감, 소속감 부족",
        "category": "person"
    },
    {
        "instruction": "Please provide a psychological interpretation of the following HTP test image caption",
        "input": "The tree has no leaves and broken branches.",
        "expected_theme": "우울, 상실감, 트라우마",
        "category": "tree"
    },
    {
        "instruction": "Please provide a psychological interpretation of the following HTP test image caption",
        "input": "A house with a large door and many windows.",
        "expected_theme": "개방성, 사회성, 소통 욕구",
        "category": "house"
    }
]

test_results = []


[2/5] 고정 테스트 케이스 평가 중...


In [6]:
# 고정 테스트 케이스 실행
for i, test in enumerate(test_cases, 1):
    print(f"\n{'='*80}")
    print(f"테스트 케이스 {i}/{len(test_cases)}: {test['category'].upper()}")
    print(f"입력: {test['input']}")
    print(f"예상 주제: {test['expected_theme']}")
    print('-' * 80)
    
    try:
        interpretation = generate_htp_interpretation(
            instruction=test['instruction'],
            image_caption=test['input'],
            max_tokens=200
        )
        
        print(f"🤖 모델 해석:\n{interpretation}")
        
        test_results.append({
            "case_id": i,
            "category": test['category'],
            "input": test['input'],
            "expected_theme": test['expected_theme'],
            "model_output": interpretation,
            "output_length": len(interpretation),
            "status": "success"
        })
        
    except Exception as e:
        print(f"❌ 생성 오류: {e}")
        test_results.append({
            "case_id": i,
            "category": test['category'],
            "input": test['input'],
            "status": "failed",
            "error": str(e)
        })

print(f"\n✅ 고정 테스트 완료: {len([r for r in test_results if r['status']=='success'])}/{len(test_cases)} 성공")


테스트 케이스 1/5: TREE
입력: The tree is dominant and tall with many branches.
예상 주제: 자신감, 성장 의지, 리더십
--------------------------------------------------------------------------------
🤖 모델 해석:
This suggests a potential need for control, assertiveness, or a desire to be perceived as strong and influential in their environment. The large size of the tree could indicate a need for power and dominance.

테스트 케이스 2/5: HOUSE
입력: A small house with no windows, located far from the center.
예상 주제: 내향성, 소통 회피, 고립감
--------------------------------------------------------------------------------
🤖 모델 해석:
This suggests a potential need for control, assertiveness, or a desire to be perceived as strong and influential in their environment. The large size of the tree could indicate a need for power and dominance.

테스트 케이스 2/5: HOUSE
입력: A small house with no windows, located far from the center.
예상 주제: 내향성, 소통 회피, 고립감
--------------------------------------------------------------------------------
🤖 모델 해석:
Th

## 4. 실제 데이터셋 샘플 테스트

In [7]:
print("\n[3/5] 실제 데이터셋 샘플 테스트 중...")

try:
    dataset = load_dataset('json', data_files=TEST_DATA_PATH)
    test_split = dataset['train'].train_test_split(test_size=0.1, seed=42)['test']
    
    print(f"   총 테스트 샘플: {len(test_split)}개")
    
    # 랜덤으로 10개 샘플 선택
    np.random.seed(42)
    sample_indices = np.random.choice(len(test_split), min(10, len(test_split)), replace=False)
    
    dataset_results = []
    
except Exception as e:
    print(f"❌ 데이터셋 로드 실패: {e}")
    dataset_results = []
    sample_indices = []


[3/5] 실제 데이터셋 샘플 테스트 중...
   총 테스트 샘플: 146개
   총 테스트 샘플: 146개


In [8]:
# 데이터셋 샘플 테스트 실행
if len(sample_indices) > 0:
    for idx, sample_idx in enumerate(sample_indices, 1):
        sample = test_split[int(sample_idx)]
        
        print(f"\n{'='*80}")
        print(f"샘플 {idx}/10 (카테고리: {sample['category']})")
        print(f"입력: {sample['input'][:100]}...")
        print('-' * 80)
        
        try:
            prediction = generate_htp_interpretation(
                instruction=sample['instruction'],
                image_caption=sample['input'],
                max_tokens=150
            )
            
            print(f"🤖 모델 예측:\n{prediction[:150]}...")
            print(f"\n📚 정답:\n{sample['output'][:150]}...")
            
            dataset_results.append({
                "sample_id": int(sample_idx),
                "category": sample['category'],
                "input": sample['input'],
                "ground_truth": sample['output'],
                "prediction": prediction,
                "input_length": len(sample['input']),
                "output_length": len(prediction),
                "status": "success"
            })
            
        except Exception as e:
            print(f"❌ 예측 실패: {e}")
            dataset_results.append({
                "sample_id": int(sample_idx),
                "status": "failed",
                "error": str(e)
            })
    
    print(f"\n✅ 데이터셋 샘플 테스트 완료: {len([r for r in dataset_results if r['status']=='success'])}/{len(sample_indices)} 성공")
else:
    print("⚠️ 테스트할 샘플이 없습니다.")


샘플 1/10 (카테고리: person)
입력: The person in the drawing has arms slightly overlapping....
--------------------------------------------------------------------------------
🤖 모델 예측:
This indicates indecision, potential conflict, or a need for reassurance. It could also signify a desire to protect oneself from external threats, sug...

📚 정답:
This finding, combined with the low scores on the neuroticism subscales (N1, N2, N3, N4), suggests a potential tendency towards interpersonal conflict...

샘플 2/10 (카테고리: tree)
입력: The tree is prominent and mature, reaching upwards....
--------------------------------------------------------------------------------
🤖 모델 예측:
This indicates indecision, potential conflict, or a need for reassurance. It could also signify a desire to protect oneself from external threats, sug...

📚 정답:
This finding, combined with the low scores on the neuroticism subscales (N1, N2, N3, N4), suggests a potential tendency towards interpersonal conflict...

샘플 2/10 (카테고리: tree)

## 5. 카테고리별 성능 분석

In [9]:
print("\n[4/5] 카테고리별 성능 분석 중...")

all_results = test_results + dataset_results
success_results = [r for r in all_results if r.get('status') == 'success']

if success_results:
    categories = {}
    for result in success_results:
        cat = result.get('category', 'unknown')
        if cat not in categories:
            categories[cat] = []
        categories[cat].append(result)
    
    print(f"\n카테고리별 통계:")
    print(f"{'카테고리':<15} {'샘플 수':<10} {'평균 출력 길이':<15}")
    print("-" * 50)
    
    for cat, results in categories.items():
        avg_length = np.mean([r['output_length'] for r in results])
        print(f"{cat:<15} {len(results):<10} {avg_length:<15.1f}")
    
    total_avg_length = np.mean([r['output_length'] for r in success_results])
    print(f"\n전체 평균 출력 길이: {total_avg_length:.1f} 자")
else:
    print("⚠️ 성공한 테스트가 없습니다.")
    categories = {}
    total_avg_length = 0


[4/5] 카테고리별 성능 분석 중...

카테고리별 통계:
카테고리            샘플 수       평균 출력 길이       
--------------------------------------------------
tree            8          293.5          
house           3          296.0          
person          4          297.5          

전체 평균 출력 길이: 295.1 자


## 6. 결과 저장

In [10]:
print("\n[5/5] 결과 저장 중...")

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1. 상세 결과 JSON
detailed_results = {
    "test_info": {
        "model_path": MODEL_PATH,
        "test_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "total_tests": len(all_results),
        "successful_tests": len(success_results),
        "success_rate": f"{100 * len(success_results) / len(all_results):.1f}%" if all_results else "0%"
    },
    "fixed_test_cases": test_results,
    "dataset_samples": dataset_results,
    "statistics": {
        "avg_output_length": float(np.mean([r['output_length'] for r in success_results])) if success_results else 0,
        "category_breakdown": {
            cat: len(results) for cat, results in categories.items()
        } if success_results else {}
    }
}

result_file = f"{OUTPUT_DIR}/test_results_{timestamp}.json"
with open(result_file, 'w', encoding='utf-8') as f:
    json.dump(detailed_results, f, indent=2, ensure_ascii=False)

print(f"✅ 상세 결과 저장: {result_file}")


[5/5] 결과 저장 중...
✅ 상세 결과 저장: ./test_results/test_results_20251115_011648.json


In [11]:
# 2. 요약 리포트 TXT
report_file = f"{OUTPUT_DIR}/test_report_{timestamp}.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write("=" * 80 + "\n")
    f.write("HTP 심리검사 해석 모델 테스트 리포트\n")
    f.write("=" * 80 + "\n\n")
    
    f.write(f"테스트 날짜: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"모델 경로: {MODEL_PATH}\n\n")
    
    f.write(f"총 테스트 수: {len(all_results)}\n")
    f.write(f"성공: {len(success_results)}\n")
    f.write(f"실패: {len(all_results) - len(success_results)}\n")
    f.write(f"성공률: {100 * len(success_results) / len(all_results):.1f}%\n\n" if all_results else "성공률: 0%\n\n")
    
    if success_results:
        f.write("카테고리별 통계:\n")
        f.write("-" * 50 + "\n")
        for cat, results in categories.items():
            avg_length = np.mean([r['output_length'] for r in results])
            f.write(f"  {cat}: {len(results)}개 샘플, 평균 {avg_length:.1f}자\n")
        
        f.write(f"\n전체 평균 출력 길이: {total_avg_length:.1f}자\n")

print(f"✅ 요약 리포트 저장: {report_file}")

✅ 요약 리포트 저장: ./test_results/test_report_20251115_011648.txt


## 최종 요약

In [12]:
print("\n" + "=" * 80)
print("테스트 완료!")
print("=" * 80)
print(f"📊 총 테스트: {len(all_results)}개")
print(f"✅ 성공: {len(success_results)}개")
print(f"❌ 실패: {len(all_results) - len(success_results)}개")
if all_results:
    print(f"📈 성공률: {100 * len(success_results) / len(all_results):.1f}%")
print(f"\n📁 결과 저장 위치: {OUTPUT_DIR}/")
print("=" * 80)


테스트 완료!
📊 총 테스트: 15개
✅ 성공: 15개
❌ 실패: 0개
📈 성공률: 100.0%

📁 결과 저장 위치: ./test_results/
